In [1]:
from analyzers.audio.audio_analyzer import AudioAnalyzer
from utils.audio_tools import extract_audio_ffmpeg
import torch
from transformers import pipeline

In [2]:
video_path = 'test.mp4'

In [3]:
audio_path = extract_audio_ffmpeg(video_path)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [4]:
audio_analyzer = AudioAnalyzer(audio_path, model_size='small')

# Получаем список сегментов с распознанным теAudioAnalyzerкстом
segments = audio_analyzer.transcribe_audio()

# Получаем значения энергии аудио для каждого временного фрейма
energy = audio_analyzer.detect_audio_activity(frame_duration=1.0)

In [10]:
scene_list = [
    {"start": item["start"], "end": item["end"], "text": item["text"]}
    for item in segments
]

In [9]:
# Используем float16, если bfloat16 не поддерживается
pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.float16,
    device_map="auto"
)

Device set to use cuda:0


In [29]:
messages = [
    {
        "role": "system",
        "content":
            "Ты — сценарист, который помогает структурировать фильм по смысловым сценам. На вход ты получаешь массив коротких сцен с таймкодами и репликами. Твоя задача — объединить их в более крупные смысловые сцены (сюжетные блоки) и выдать результат в виде массива объектов со значениями `start`, `end` и кратким `summary` (описанием происходящего). Не пропускай сцены. Объединяй их только по смыслу. Таймкоды в секундах. Пример вывода:\n"
            "[{'start': 0.0, 'end': 24.0, 'summary': 'Общий смысл'}, ...]"

    },
    {
        "role": "user",
        "content":
            "Вот список сцен:\n\n" +
            str(scene_list) +
            "\n\nСгруппируй их в смысловые блоки и выведи новый массив с полями `start`, `end`, `summary`."

    }
]

In [30]:
# Сформируем промпт
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [31]:
# Запустим генерацию
outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

In [32]:
# Распечатаем результат
print(outputs[0]["generated_text"])

<|system|>
Ты — сценарист, который помогает структурировать фильм по смысловым сценам. На вход ты получаешь массив коротких сцен с таймкодами и репликами. Твоя задача — объединить их в более крупные смысловые сцены (сюжетные блоки) и выдать результат в виде массива объектов со значениями `start`, `end` и кратким `summary` (описанием происходящего). Не пропускай сцены. Объединяй их только по смыслу. Таймкоды в секундах. Пример вывода:
[{'start': 0.0, 'end': 24.0, 'summary': 'Общий смысл'}, ...]</s>
<|user|>
Вот список сцен:

[{'start': 0.0, 'end': 5.6000000000000005, 'text': ' Рванули в Россия за славой. В армии у нас кто угодно. Немцы, итальяны, даже шавайцары.'}, {'start': 5.6000000000000005, 'end': 7.6000000000000005, 'text': ' 600 тысяч штыков.'}, {'start': 16.1, 'end': 17.6, 'text': ' Естественно, что за дерьмо!'}, {'start': 17.6, 'end': 22.0, 'text': ' Месье Наполеон, ты нам дороги на карте рисовали, а дорог ни хера нет.'}, {'start': 22.0, 'end': 24.0, 'text': ' Наебал получается.